In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

model_path = "clasificador_analisis/clasificador/clasificador_tono/classweights_focalloss/comparativa/ep_3_gamma_0.30/modelo_final"


tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)


In [5]:
def predict_sentiment_with_neutral(text, model, tokenizer, alpha=1.0, neutral_threshold=0.1):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probs = torch.softmax(logits, dim=1).squeeze()

    neutral_score = 1 - (abs(probs[0] - 0.5) ** alpha + abs(probs[2] - 0.5) ** alpha)

    if neutral_score > (1 - neutral_threshold):
        return 1  
    else:
        return int(torch.argmax(probs))


In [ ]:
df = pd.read_excel("clasificador_analisis/clasificador/clasificador_tono/classweights_focalloss/mejor_test.xlsx")
df = df.rename(columns={"text": "text", "label": "label_manual_num"})

df["label_predicted"] = df["text"].apply(lambda x: predict_sentiment_with_neutral(x, model, tokenizer))

print("\n📊 MATRIZ DE CONFUSIÓN:")
print(confusion_matrix(df["label_manual_num"], df["label_predicted"]))

print("\n📈 CLASIFICATION REPORT:")
print(classification_report(
    df["label_manual_num"],
    df["label_predicted"],
    target_names=["Negativo", "Neutro", "Positivo"],
    digits=3
))


📊 MATRIZ DE CONFUSIÓN:
[[212  15  11]
 [ 20  58  11]
 [ 14   4 170]]

📈 CLASIFICATION REPORT:
              precision    recall  f1-score   support

    Negativo      0.862     0.891     0.876       238
      Neutro      0.753     0.652     0.699        89
    Positivo      0.885     0.904     0.895       188

    accuracy                          0.854       515
   macro avg      0.833     0.816     0.823       515
weighted avg      0.852     0.854     0.852       515

